In [25]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from statsmodels.stats.multitest import multipletests


In [86]:
REPLICATION = True

file_path_add = ''
if REPLICATION:
    file_path_add = 'replication/'

basemodelname = 'gpt2'

In [87]:
models = ['country_based','topic based','random']


In [88]:
index = 'token_pairs'

## Evaluating on WVS

In [89]:
list_rows = []
for model in models:
    pew_gpt2 = pd.read_csv(f'../data/{file_path_add}wvs_w7_{basemodelname}_{model}_on_pew_{index}_in.csv')
    pew_gpt2_cultural = pew_gpt2.loc[pew_gpt2['country'] != 'universal']
    pew_gpt2_cultural = pew_gpt2_cultural.loc[~pd.isna(pew_gpt2_cultural['wvs_score'])]
    r, p = (scipy.stats.pearsonr(pew_gpt2_cultural['wvs_score'], pew_gpt2_cultural['log prob difference']))
    row = {'model':basemodelname,'train_data' : 'PEW', 'eval_data': 'WVS',
          'strategy': model, 'r': r, 'p': p, 'n': len(pew_gpt2_cultural)}
    list_rows.append(row)


In [90]:
df = pd.DataFrame(list_rows)
df['p'] = multipletests(df['p'], method = 'bonferroni', alpha = 0.5)[1]
df

,model,train_data,eval_data,strategy,r,p,n
0,gpt2,PEW,WVS,country_based,0.429615,6.026195e-47,1028
1,gpt2,PEW,WVS,topic based,0.426214,3.778737e-46,1028
2,gpt2,PEW,WVS,random,0.458814,3.539759e-54,1028


## Evaluating on PEW
we have to remove seen pairs from evaluation set

In [91]:
pew_topics = {'use contraceptives':'using contraceptives',
           'get a divorce':'getting a divorce', 
            'have an abortion': 'having an abortion',
            'be homosexual': 'homosexuality', 
           'drink alcohol': 'drinking alcohol',
           'have an extramarital affair': 'married people having an affair' ,
             'gamble': 'gambling',
       'have sex between unmarried adults':'sex between unmarried adults'
                         }
pew_topics_list = list(pew_topics.keys())



In [92]:
def included_function(pairs):
    def func(row):
        return (row['country'], pew_topics[row['topic']]) in pairs
    return func

In [93]:
all_eval_pairs = pickle.load(open(f'../data/{file_path_add}pew_eval_pairs.p', 'rb'))
list_rows = []
for model in models:
    eval_pairs = all_eval_pairs[model]
    pew_gpt2 = pd.read_csv(f'../data/{file_path_add}pew_{basemodelname}_{model}_on_pew_{index}.csv')
    pew_gpt2_cultural = pew_gpt2.loc[pew_gpt2['country'] != 'universal']
    
    pew_gpt2_cultural = pew_gpt2_cultural.loc[~pd.isna(pew_gpt2_cultural['pew_score'])]
    pew_gpt2_cultural['in_eval'] = pew_gpt2_cultural.apply(included_function(eval_pairs), axis = 1)
    pew_gpt2_cultural = pew_gpt2_cultural.loc[pew_gpt2_cultural.in_eval == True]

    r, p = scipy.stats.pearsonr(pew_gpt2_cultural['pew_score'], pew_gpt2_cultural['log prob difference'])
    row = {'model':basemodelname,'train_data' : 'PEW', 'eval_data': 'PEW',
          'strategy': model, 'r': r, 'p': p, 'n': len(pew_gpt2_cultural)}
    list_rows.append(row)

In [94]:
df = pd.DataFrame(list_rows)
df['p'] = multipletests(df['p'], method = 'bonferroni', alpha = 0.5)[1]
df

,model,train_data,eval_data,strategy,r,p,n
0,gpt2,PEW,PEW,country_based,0.727773,2.872348e-11,64
1,gpt2,PEW,PEW,topic based,0.650598,3.414898e-10,78
2,gpt2,PEW,PEW,random,0.832700,7.981025e-17,63


# Variation study

### WVS

In [70]:
topics = ['claiming government benefits to which you are not entitled',
 'avoiding a fare on public transport',
 'stealing property',
 'cheating on taxes',
 'someone accepting a bribe in the course of their duties',
 'homosexuality',
 'prostitution',
 'abortion',
 'divorce',
 'sex before marriage',
 'suicide',
 'euthanasia',
 'for a man to beat his wife',
 'parents beating children',
 'violence against other people',
 'terrorism as a political, ideological or religious mean',
 'having casual sex',
 'political violence',
 'death penalty']

In [71]:
variation_rows = []

for i, model in enumerate(models):
    if model == 'topic based':
        model_refined = 'topic_based'
    elif model == 'removed topics':
        model_refined = 'removed_topic'
    else:
        model_refined = model
    wvs_gpt2 = pd.read_csv(f'../data/{file_path_add}wvs_w7_{basemodelname}_{model}_on_pew_{index}_in.csv')
    wvs_gpt2_cultural = wvs_gpt2.loc[wvs_gpt2['country'] != 'universal']
    df = wvs_gpt2_cultural.loc[~pd.isna(wvs_gpt2_cultural['wvs_score'])]

    for t in topics:

        pew_gpt2_t = df.loc[df.topic == t]
        user_var = np.var(pew_gpt2_t['wvs_score'])
        model_var = np.var(pew_gpt2_t['log prob difference'])
        row = {'model': model, 'user variation': user_var, 'model variation': model_var, 'topic': t}
        variation_rows.append(row)

df = pd.DataFrame(variation_rows)

   


In [72]:
list_rows = []
for i, model in enumerate(models):
    
    model_df = df.loc[df.model == model] 
    r, p = scipy.stats.pearsonr(model_df['model variation'],model_df['user variation'])
    
    row = {'strategy': model, 'r': r, 'p':p,
          'n': len(model_df)}
    list_rows.append(row)

In [73]:
df = pd.DataFrame(list_rows)
df['p'] = multipletests(df['p'], method = 'bonferroni', alpha = 0.5)[1]
df

,strategy,r,p,n
0,country_based,0.775878,0.000284,19
1,topic based,0.612060,0.016046,19
2,random,0.696045,0.002800,19


## PEW

In [74]:
pew_topics = {'using contraceptives':['using contraceptives', 'use contraceptives'],
           'getting a divorce': ['getting a divorce','get a divorce'], 
           'having an abortion': ['having an abortion','have an abortion'],
           'homosexuality': ['homosexuality','be homosexual'], 
           'drinking alcohol': ['drinking alcohol','drink alcohol'],
           'married people having an affair' : ['married people having an affair', 'have an extramarital affair'],
            'gambling': ['gambling','gamble'],
       'sex between unmarried adults': [ 'sex between unmarried adults','have sex between unmarried adults']
                         }

pew_topics_list = list(pew_topics.keys())

variation_rows = []

for i, model in enumerate(models):
    if model == 'topic based':
        model_refined = 'topic_based'
    elif model == 'removed topics':
        model_refined = 'removed_topic'
    else:
        model_refined = model
    pew_gpt2 = pd.read_csv(f'../data/{file_path_add}pew_{basemodelname}_{model}_on_pew_{index}.csv')
    pew_gpt2_cultural = pew_gpt2.loc[wvs_gpt2['country'] != 'universal']
    df = pew_gpt2_cultural.loc[~pd.isna(pew_gpt2_cultural['pew_score'])]

    for t in pew_topics:

        pew_gpt2_t = df.loc[df.topic.isin(pew_topics[t])]
        user_var = np.var(pew_gpt2_t['pew_score'])
        model_var = np.var(pew_gpt2_t['log prob difference'])
        row = {'model': model, 'user variation': user_var, 'model variation': model_var, 'topic': t}
        variation_rows.append(row)

df = pd.DataFrame(variation_rows)

   




In [75]:
list_rows = []
for i, model in enumerate(models):
    
    model_df = df.loc[df.model == model]
    

    slope, intercept, r, p, std_err =\
    scipy.stats.linregress(model_df['model variation'],model_df['user variation'])
    
    
    row = {'strategy': model, 'r': r, 'p':p,'slope':slope,
          'n': len(model_df)}
    list_rows.append(row)
    


In [76]:
df = pd.DataFrame(list_rows)
df['p'] = multipletests(df['p'], method = 'bonferroni', alpha = 0.5)[1]
df

,strategy,r,p,slope,n
0,country_based,0.929280,0.002514,0.563220,8
1,topic based,0.848208,0.023335,0.677774,8
2,random,0.905824,0.005830,0.363514,8
